# Shap speed analysis
- analysis uses the following CPU and Wall clock metrics:
https://stackoverflow.com/questions/56672370/how-to-capture-wall-clock-time-and-cpu-time-in-a-python-variable-using-bash-buil 

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import json
import numpy as np
import os

/dfs/scratch1/gmachi/miniconda3/envs/kkenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
test = "/dfs/scratch1/gmachi/datasets/WikiSection/wikisection_en_disease_test.json"
with open(test, 'r') as f:
    test_data = json.load(f)

save_path = "/dfs/scratch1/gmachi/datasets/wikisection_processed/"
# save_path_attn = os.path.join(save_path, "attn_Gs")
# save_path_entail = os.path.join(save_path, "entail_Gs")
# save_path_prob = os.path.join(save_path, "prob_Gs")
# save_path_lime = os.path.join(save_path, "lime_Gs")
save_path_shap = os.path.join(save_path, "shap_Gs")
save_path_shap_fc = os.path.join(save_path, "shap_fullcontext")


target = "disease.genetics"  # class-1

In [42]:
len(test_data)

718

In [4]:
from utils import process_sentences
from utils import serialize, deserialize
from utils import convert_text2graph
import random
import networkx as nx

In [5]:
def chunker(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]

In [6]:
from time import process_time 
from time import perf_counter

below is for full-context 

In [38]:
# def process_Zs_scores(test_data, score_fn, save_path_Gs, batch_size=1, kill_after=2):
#     dt_cpu_data, dt_cpu_batches, dt_cpu_tokens = [], [], []
#     dt_wall_data, dt_wall_batches, dt_wall_tokens = [], [], []
#     finished = 0
#     ran_idxs = []
    
#     for i in range(len(test_data)):
#         if finished == kill_after:
#             return ran_idxs, dt_cpu_tokens, dt_wall_tokens, dt_cpu_batches, dt_wall_batches, dt_cpu_data, dt_wall_data

#         G_file = "doc_"+str(i) + "_graph.obj"
#         save_path_G = os.path.join(save_path_Gs, G_file)
#         if os.path.isfile(save_path_G) == True:
#             print("skipping sample bc already created:", G_file)
#             continue

#         text = test_data[i]["text"]
#         annots = test_data[i]["annotations"]

#         scores = []
#         doc_sents = []
#         for annot in annots:
#             begin = int(annot["begin"])
#             idx = [begin, begin+int(annot["length"])]
#             chunk = text[idx[0]:idx[1]]
#             sents = [s for s in chunk.split(".")]
#             sents = process_sentences(sents)
#             doc_sents.extend(sents)
        
#         doc = [" ".join(doc_sents)]
#         print(len(doc[0].split(" ")))

#         #===============================
#         start_cpu_datum = process_time()
#         start_wall_datum = perf_counter()
#         dt_cpu_batch, dt_cpu_token = [], []
#         dt_wall_batch, dt_wall_token = [], []
    

#         # all just in one pass
#         #========================
#         start_cpu_batch = process_time()  
#         start_wall_batch = perf_counter()
#         es = score_fn(doc)
#         stop_cpu_batch = process_time() 
#         stop_wall_batch = perf_counter()
#         #========================
#         tdb_cpu = float(stop_cpu_batch - start_cpu_batch) # batch
#         tdb_wall = float(stop_wall_batch - start_wall_batch) # batch
#         print("dt cpu -> batch:", tdb_cpu)
#         print("dt wall -> batch:", tdb_wall)
#         dt_cpu_batch.append(tdb_cpu)
#         dt_wall_batch.append(tdb_wall)

#         scores.append(es)

#         stop_cpu_datum = process_time()  
#         stop_wall_datum = perf_counter() 


#         #==============================
#         tdd_cpu = float(stop_cpu_datum - start_cpu_datum) # datum
#         tdd_wall = float(stop_wall_datum - start_wall_datum) # datum
#         print("dt cpu -> datum:", tdd_cpu)
#         print("dt wall -> datum:", tdd_wall)
#         dt_cpu_data.append(tdd_cpu)
#         dt_wall_data.append(tdd_wall)
        
#         dt_cpu_tokens.append(dt_cpu_token)
#         dt_wall_tokens.append(dt_wall_token)
#         dt_cpu_batches.append(dt_cpu_batch)
#         dt_wall_batches.append(dt_wall_batch)
        
#         # save
#         Z = np.array(scores)
#         G = convert_text2graph(Z) # save as graph
#         serialize(G, save_path_G)
#         finished += 1
#         print("finished:", finished)
#         ran_idxs.append(i)
#         xx

#     return ran_idxs, dt_cpu_tokens, dt_wall_tokens, dt_cpu_batches, dt_wall_batches, dt_cpu_data, dt_wall_data

In [39]:
# from text_baselines import deberta_shapZSC_fullcontext
# outs = process_Zs_scores(test_data, deberta_shapZSC_fullcontext, save_path_shap_fc)

skipping sample bc already created: doc_0_graph.obj
skipping sample bc already created: doc_1_graph.obj
2354


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
 10%|█         | 50/498 [00:40<07:29,  1.00s/it]/dfs/scratch1/gmachi/miniconda3/envs/kkenv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
PartitionExplainer explainer: 2it [09:40, 580.03s/it]              


dt cpu -> batch: 584.0735837940001
dt wall -> batch: 582.1689849235117
dt cpu -> datum: 584.0738073730001
dt wall -> datum: 582.1691266559064
finished: 1


NameError: name 'xx' is not defined

inspect the graph quickly

In [7]:
from utils import deserialize
G = "/dfs/scratch1/gmachi/datasets/wikisection_processed/shap_Gs/doc_22_graph.obj"
G = deserialize(G)
len(G.nodes)

4

below is for partial context SHAP saved at shap_Gs

In [51]:
def process_Zs_scores(test_data, score_fn, save_path_Gs, batch_size=1, kill_after=2):
    dt_cpu_data, dt_cpu_batches, dt_cpu_tokens = [], [], []
    dt_wall_data, dt_wall_batches, dt_wall_tokens = [], [], []
    finished = 0
    ran_idxs = []
    
    for i in range(len(test_data)):
        if finished == kill_after:
            return ran_idxs, dt_cpu_tokens, dt_wall_tokens, dt_cpu_batches, dt_wall_batches, dt_cpu_data, dt_wall_data

        G_file = "doc_"+str(i) + "_graph.obj"
        save_path_G = os.path.join(save_path_Gs, G_file)
        if os.path.isfile(save_path_G) == True:
            print("skipping sample bc already created:", G_file)
            continue

        text = test_data[i]["text"]
        annots = test_data[i]["annotations"]

        scores = []
        doc_sents = []
        for annot in annots:
            begin = int(annot["begin"])
            idx = [begin, begin+int(annot["length"])]
            chunk = text[idx[0]:idx[1]]
            sents = [s for s in chunk.split(".")]
            sents = process_sentences(sents)
            doc_sents.extend(sents)

        #===============================
        start_cpu_datum = process_time()
        start_wall_datum = perf_counter()
        dt_cpu_batch, dt_cpu_token = [], []
        dt_wall_batch, dt_wall_token = [], []
        for batch in chunker(doc_sents, batch_size):
            #========================
            start_cpu_batch = process_time()  
            start_wall_batch = perf_counter()
            es = score_fn(batch)
            stop_cpu_batch = process_time() 
            stop_wall_batch = perf_counter()
            #========================
            tdb_cpu = float(stop_cpu_batch - start_cpu_batch) # batch
            tdb_wall = float(stop_wall_batch - start_wall_batch) # batch
            print("dt cpu -> batch:", tdb_cpu)
            print("dt wall -> batch:", tdb_wall)
            dt_cpu_batch.append(tdb_cpu)
            dt_wall_batch.append(tdb_wall)

            tdt_cpu = float(tdb_cpu / batch_size) # token
            tdt_wall = float(tdb_wall / batch_size) # token
            print("dt cpu -> token:", tdt_cpu)
            print("dt wall -> token:", tdt_wall)
            dt_cpu_token.append(tdt_cpu)
            dt_wall_token.append(tdt_wall)

            scores.extend(es)

        stop_cpu_datum = process_time()  
        stop_wall_datum = perf_counter() 
        #==============================
        tdd_cpu = float(stop_cpu_datum - start_cpu_datum) # datum
        tdd_wall = float(stop_wall_datum - start_wall_datum) # datum
        print("dt cpu -> datum:", tdd_cpu)
        print("dt wall -> datum:", tdd_wall)
        dt_cpu_data.append(tdd_cpu)
        dt_wall_data.append(tdd_wall)
        
        dt_cpu_tokens.append(dt_cpu_token)
        dt_wall_tokens.append(dt_wall_token)
        dt_cpu_batches.append(dt_cpu_batch)
        dt_wall_batches.append(dt_wall_batch)
        
        # save
        Z = np.array(scores)
        G = convert_text2graph(Z) # save as graph
        serialize(G, save_path_G)
        finished += 1
        print("finished:", finished)
        ran_idxs.append(i)

    return ran_idxs, dt_cpu_tokens, dt_wall_tokens, dt_cpu_batches, dt_wall_batches, dt_cpu_data, dt_wall_data

In [52]:
from text_baselines import deberta_shapZSC
outs = process_Zs_scores(test_data, deberta_shapZSC, save_path_shap)

skipping sample bc already created: doc_0_graph.obj
skipping sample bc already created: doc_1_graph.obj
skipping sample bc already created: doc_2_graph.obj
skipping sample bc already created: doc_3_graph.obj
skipping sample bc already created: doc_4_graph.obj
skipping sample bc already created: doc_5_graph.obj
skipping sample bc already created: doc_6_graph.obj
skipping sample bc already created: doc_7_graph.obj
skipping sample bc already created: doc_8_graph.obj
skipping sample bc already created: doc_9_graph.obj
skipping sample bc already created: doc_10_graph.obj
skipping sample bc already created: doc_11_graph.obj
skipping sample bc already created: doc_12_graph.obj
skipping sample bc already created: doc_13_graph.obj
skipping sample bc already created: doc_14_graph.obj
skipping sample bc already created: doc_15_graph.obj
skipping sample bc already created: doc_16_graph.obj
skipping sample bc already created: doc_17_graph.obj
skipping sample bc already created: doc_18_graph.obj


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


dt cpu -> batch: 11.422503930999937
dt wall -> batch: 5.049421355128288
dt cpu -> token: 11.422503930999937
dt wall -> token: 5.049421355128288


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
PartitionExplainer explainer: 2it [00:18, 18.49s/it]               


dt cpu -> batch: 27.066750195000168
dt wall -> batch: 20.64337133616209
dt cpu -> token: 27.066750195000168
dt wall -> token: 20.64337133616209


...

dt cpu -> batch: 9.62351777099957
dt wall -> batch: 3.2688243240118027
dt cpu -> token: 9.62351777099957
dt wall -> token: 3.2688243240118027


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


dt cpu -> batch: 8.829161388000102
dt wall -> batch: 2.457034356892109
dt cpu -> token: 8.829161388000102
dt wall -> token: 2.457034356892109


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
PartitionExplainer explainer: 2it [00:12, 12.09s/it]               


dt cpu -> batch: 20.5306543480001
dt wall -> batch: 14.113164123147726
dt cpu -> token: 20.5306543480001
dt wall -> token: 14.113164123147726
dt cpu -> datum: 1953.3149264129997
dt wall -> datum: 1438.068976353854
finished: 2


In [54]:
from utils import serialize
import os

save_path = "/dfs/scratch1/gmachi/k2/K2/src/outputs/shapley_wallclock"
filename = '-'.join(str(x) for x in outs[0]) + "_wallclock.obj"
save_path = os.path.join(save_path, filename)
serialize(outs, save_path)

In [55]:
from utils import deserialize
outs_now = deserialize(save_path)
print(outs_now)

([19, 20], [[11.422503930999937, 27.066750195000168, 21.895460878999984, 10.428797032000148, 26.794675280999854, 26.394105472999854, 12.044467488999999, 8.454951615000027, 26.59015132400009, 26.899714383000173], [26.39106044900018, 26.815743083000143, 26.9129290879996, 24.217442632000257, 26.69086101699986, 15.168089093999697, 26.839411875999758, 11.813787477999995, 25.872177381000256, 15.230490432000352, 18.11643864200005, 19.416962737000176, 26.89829607999991, 26.503636818999894, 27.013590913999906, 27.102540455000053, 24.13822394099998, 21.546188739000172, 26.401084966000326, 26.002959974000078, 26.963183630999993, 12.968358167999668, 26.984214089000034, 26.167411025000092, 26.929788671000097, 19.371963587999744, 25.307164229999216, 15.346762559000126, 19.76951429700057, 22.941591248000805, 26.651758328999676, 15.68178748199989, 19.348809383000116, 24.441241624999748, 26.875570733999666, 27.476202953999746, 27.7996515800005, 18.368994196999665, 30.213020256999698, 29.766381718000048

# analysis of speed benchmarking

In [56]:
from collections.abc import Iterable

def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [57]:
from utils import deserialize
import os

load_path = "/dfs/scratch1/gmachi/k2/K2/src/outputs/shapley_wallclock"
cts,wts = [],[]
cbs,wbs = [],[]
cds,wds = [],[]

for f in os.listdir(load_path):
    f = os.path.join(load_path, f)
    if os.path.isfile(f) == False:
        continue
    _, dt_cpu_tokens, dt_wall_tokens, dt_cpu_batches, dt_wall_batches, dt_cpu_data, dt_wall_data = deserialize(f)
    cts.extend([el for el in flatten(dt_cpu_tokens)])
    wts.extend([el for el in flatten(dt_wall_tokens)])
    cbs.extend([el for el in flatten(dt_cpu_batches)])
    wbs.extend([el for el in flatten(dt_wall_batches)])
    cds.extend([el for el in flatten(dt_cpu_data)])
    wds.extend([el for el in flatten(dt_wall_data)])
    

In [58]:
print(cts)

[11.175186075000056, 20.02465968200022, 26.296580453999923, 15.262312691999796, 18.12888737100002, 12.915355073999763, 27.049799595999957, 13.448487327999828, 13.202001183999982, 11.828293497999766, 19.550307879999764, 26.721630146999814, 25.32021292399986, 8.610786585999904, 13.69562560400027, 14.907470034000198, 26.641327655999703, 14.598418741999922, 15.371750958999655, 26.74878920099991, 26.891111407000153, 26.775289539999903, 16.20220842000026, 26.427380318999894, 17.991241971999898, 10.799475050999717, 13.310504754999783, 16.55602471200018, 23.142870505000246, 26.446412194000004, 26.23179778199983, 17.878696905000197, 26.211791702000028, 11.422503930999937, 27.066750195000168, 21.895460878999984, 10.428797032000148, 26.794675280999854, 26.394105472999854, 12.044467488999999, 8.454951615000027, 26.59015132400009, 26.899714383000173, 26.39106044900018, 26.815743083000143, 26.9129290879996, 24.217442632000257, 26.69086101699986, 15.168089093999697, 26.839411875999758, 11.81378747799

In [59]:
import numpy as np
print(np.mean(cts), np.std(cts))
print(np.mean(wts), np.std(wts))
# print(np.mean(cbs), np.std(cbs))
# print(np.mean(wbs), np.std(wbs))
print(np.mean(cds), np.std(cds))
print(np.mean(wds), np.std(wds))

23.44745769327038 5.959631378150818
17.118726518452807 5.917034142286063
1092.658872448 1026.4095977351803
797.7381456665695 768.5140014901095


In [43]:
G = "/dfs/scratch1/gmachi/datasets/wikisection_processed/shap_Gs/doc_20_graph.obj"
G = deserialize(G)
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81))